### 人才价格计算器
   - 输入：求职者——求职职位、工作年限、学历；招聘方——公司规模
   - 输出：在指定公司规模、求职职位和经验、学历的条件下，市场水平的月薪均值
  
   - 原理：KNN分类器改进版（改进输出为离散值）——认为离输入x最近的k个点对应的月薪均值的均值是x的输出
        - k默认为训练样本数/100+1
        - 距离度量为欧氏距离
        - 决策规则：加权距离最近的k个点的y的均值
            - 加权距离：根据不同岗位下，学历、经验、公司规模与薪水的相关系数，计算输入x与训练集x的加权距离
            - y的均值：为了改进输出为离散值的问题，认为离输入x最近的k个点对应的月薪均值的均值是x的输出
   
   - 不足：
       - 输入的工作年限为个体值，但用于计算距离的工作年限为范围均值，会导致在范围交界点的个体值错误归类
           - 改进：将输入的个体值映射到各个工作年限要求范围均值上去(已改进)
       - 没有将职位描述这个信息量丰富的长文本纳入建模
           - 改进：可加入职位描述关键词与求职者能力关键词匹配程度，影响月薪水平
       - 训练样本太少，总共只有2398条数据，分岗位来看就更少了，因为样本量不足，会导致预测结果不准确
           - 改进：从boss、猎聘等网站拓展样本数量

In [9]:
import pandas as pd
import numpy as np
import operator

In [13]:
# 人才价格计算器

def KNNClassify(newinput, dataset, labels, data,position,k=None):    
    # 若未设定k值，则默认为10
    if k==None:
        k=10

    # 由于每个职位各因素与薪水相关性不同，因此引入不同职位的相关系数，放入距离加权计算中
    # 计算不同岗位类型下，公司规模、经验、学历与薪水的相关系数，用于后面计算k个最近点的加权距离
    corr=data[data.classified_zw==position][['salary_mean','companysize_mean','workyear_mean','degree_num']].corr()
    corr=np.array(corr)
    
    #salary_companysize=corr[1][0]
    #salary_workyear=corr[2][0]
    #salary_degree=corr[3][0]
    weight=np.array([corr[1][0],corr[2][0],corr[3][0]])
    
    numSamples=dataset.shape[0]
    """step1: 计算待分类数据与训练集各数据点的距离（欧氏距离：距离差值平方和开根号）"""
    diff=np.tile(newinput,(numSamples,1)) - dataset # 凸显numpy数组的高效性——元素级的运算
    squaredist=(diff**2)/weight  # 相关系数越大，因素越重要，表现为加权距离小，所以要除以相关系数
    distance = (squaredist.sum(axis=1))**0.5 # axis=1,按行累加
    
    """step2：将距离按升序排序，并取距离最近的k个近邻点"""
    # 对数组distance按升序排序，返回数组排序后的值对应的索引值
    sortedDistance=distance.argsort() 
    
    # 定义一个空字典，存放k个近邻点的分类计数
    classCount={}
    
    # 对k个近邻点分类计数，多数表决法
    for i in range(k):
        # 第i个近邻点在distance数组中的索引,对应的分类
        votelabel=labels[sortedDistance[i]]
        
        # votelabel作为字典的key，对相同的key值累加（多数表决法）
        classCount[votelabel]=classCount.get(votelabel,0)+1 
     
    # 求k个近邻点的y值的均值，作为新输入x的预测输出y
    predict=0
    point_k=0
    for key,value in classCount.items():
        predict+=key*value
        point_k+=value
    predict=predict/point_k
    
    return position,predict


# data:清洗后的数据集，如拉勾成都地区招聘岗位的数据，lagou_origin。这里对于data要求比较死，如果更换数据，需要维护代码
# position:求职职位：数据分析师,算法工程师，java工程师、等
# newinput=["companysize_mean","workyear_mean","degree_num"]
def talent_calculator(data,k=None):

    # 交互，用户输入职位、公司规模、经验、学历
    print ("可选职位：java工程师、前端、数据分析师、算法工程师")
    print ("产品运营、测试工程师、产品经理、数据挖掘、建模工程师、爬虫工程师、产业研究员职位也可计算，但由于数据量较小，结果准确性差")
    position=str(input("请输入职位："))
    companysize=float(input("请输入公司规模(人)："))
    workyear=float(input("请输入求职者工作年限(年)："))
    degree=float(input("请输入求职者学历(0-不限,1-大专,2-本科,3-硕士,4-博士)："))
    
    # 将输入的个体值映射到各个工作年限要求范围均值上去(避免在范围交界点的工作年限归类错误)
    # 若未输入工作年限，则默认为0年
    if workyear==None:
        workyear=0
    elif workyear>0 and workyear<1:
        workyear=0.5
    elif workyear>=1 and workyear<3:
        workyear=2
    elif workyear>=3 and workyear<5:
        workyear=4
    elif workyear>=5 and workyear<10:
        workyear=7.5
    elif workyear>=10:
        workyear=10
    else:
        workyear=workyear
  
    newinput=[companysize,workyear,degree]
    
    # 建立训练集
    data_x=data[data.classified_zw==position].loc[:,["companysize_mean","workyear_mean","degree_num"]]
    data_y=data[data.classified_zw==position].loc[:,["salary_mean"]]

    np.random.seed(7)
    indices=np.random.permutation(len(data_x))

    data_x_train=np.array(data_x.iloc[indices])
    data_y_train=np.array(data_y.iloc[indices])

    data_y_train.shape=(len(data_x),)
    
    dataset=data_x_train
    labels=data_y_train
    
    # 若未设定k值，则默认为输入岗位的职位数量/100+1
    if k==None:
        k=int(len(data_x)/100)+1
    
    # KNN分类器（改进版）
    # 由于每个职位各因素与薪水相关性不同，因此引入不同职位的相关系数，放入距离加权计算中
    # 计算不同岗位类型下，公司规模、经验、学历与薪水的相关系数，用于后面计算k个最近点的加权距离
    corr=data[data.classified_zw==position][['salary_mean','companysize_mean','workyear_mean','degree_num']].corr()
    corr=np.array(corr)
    
    #salary_companysize=corr[1][0]
    #salary_workyear=corr[2][0]
    #salary_degree=corr[3][0]
    weight=np.array([corr[1][0],corr[2][0],corr[3][0]])
    
    numSamples=dataset.shape[0]
    """step1: 计算待分类数据与训练集各数据点的距离（欧氏距离：距离差值平方和开根号）"""
    diff=np.tile(newinput,(numSamples,1)) - dataset # 凸显numpy数组的高效性——元素级的运算
    squaredist=(diff**2)/weight  # 相关系数越大，因素越重要，表现为加权距离小，所以要除以相关系数
    distance = (squaredist.sum(axis=1))**0.5 # axis=1,按行累加
    
    """step2：将距离按升序排序，并取距离最近的k个近邻点"""
    # 对数组distance按升序排序，返回数组排序后的值对应的索引值
    sortedDistance=distance.argsort() 
    
    # 定义一个空字典，存放k个近邻点的分类计数
    classCount={}
    
    # 对k个近邻点分类计数，多数表决法
    for i in range(k):
        # 第i个近邻点在distance数组中的索引,对应的分类
        votelabel=labels[sortedDistance[i]]
        
        # votelabel作为字典的key，对相同的key值累加（多数表决法）
        classCount[votelabel]=classCount.get(votelabel,0)+1 
     
    # 求k个近邻点的y值的均值，作为新输入x的预测输出y
    predict=0
    point_k=0
    for key,value in classCount.items():
        predict+=key*value
        point_k+=value
    predict=round(predict/point_k,2)
    
    print ("\n求职岗位为:",position,"\n人才价格为",predict,"K")
    
    return position,predict

#### 使用人才价格计算器

In [18]:
# 载入原始数据集
lagou_orgin=pd.read_csv(r"E:\python\data\lagou\lagou2018_chuli.csv",encoding="utf-8",delimiter="\t")
# 填入原始数据集后，调用人才价格计算器
talent_calculator(lagou_orgin)

可选职位：java工程师、前端、数据分析师、算法工程师
产品运营、测试工程师、产品经理、数据挖掘、建模工程师、爬虫工程师、产业研究员职位也可计算，但由于数据量较小，结果准确性差
请输入职位：数据分析师
请输入公司规模(人)：60
请输入求职者工作年限(年)：5
请输入求职者学历(0-不限,1-大专,2-本科,3-硕士,4-博士)：2

求职岗位为: 数据分析师 
人才价格为 22.5 K


('数据分析师', 22.5)